In [1]:
import Pkg; 
Pkg.activate("Conf_Fluids")
Pkg.add("Symbolics")
using Symbolics
using StaticArrays

 Activating environment at `~/Julia/Fluidos_Conformes/Conf_Fluids/Project.toml`
   Updating registry at `~/.julia/registries/General`
######################################################################### 100.0%
  Resolving package versions...
No Changes to `~/Julia/Fluidos_Conformes/Conf_Fluids/Project.toml`
No Changes to `~/Julia/Fluidos_Conformes/Conf_Fluids/Manifest.toml`


In [53]:
function F(flu,con,χ)
    f = Dict(:μ => 1, :v => 2, :x1 => 3, :x2 => 4,:x3 => 5) #para acordarse de que campo asignamos a cada componente del vector
    c = Dict(:e => 1, :s => 2, :c1 => 3, :c2 => 4,:c3 => 5)
#@variables flu[1:N], con[1:N], χ[1:3] #variables para manipulación algebraica
#@variables χ₀, χ₁, χ₂
#@variables 

    μ = flu[f[:μ]]  # esto es -μ
    #μ = view(flu,1)
    T = (μ)^(-1//2) # use μ positive, so I changed μ -> -μ
    v = flu[f[:v]]
    χ₀ = χ[1]
    χ₁ = χ[2]
    χ₂ = χ[3]
    x1 = flu[f[:x1]]
    x2 = flu[f[:x2]]
    x3 = flu[f[:x3]]
    γ = (1 - v^2)^(-1//2)
    τ = -2χ₁ * x3 * T / (γ*μ^3) - 24χ₂*(1//2*(1-v^2)x3^2 + 14//3 * x2^2 + 7/5*(1-v^2)x1*x3)/μ^5
    ρ = -6χ₀ / μ^2 - 6χ₁*x1/(γ * μ^4 * T) - 42χ₂*(6//5 *x1^2 + 10γ^2*x2^2 + 3//2*(v^2-1)^2*x3^2)/(μ^5 * γ^2)
    Q = -10χ₀ * x2 * T / μ^3 - 168χ₂ * x2 * (x1 - (v^2 - 1)x3)/(γ * μ^5)

    A = (-12 * χ₂/μ^4)*[3(2γ^2 - 1)  3v*(6γ^2 - 1)    3v^2 ;
                v*(6γ^2 - 1)  (6γ^2*(1 + 2v^2) - 1)  v*(v^2 + 2) ;
                (6γ^2 * v^2 + 1)  v*(6γ^2*(2 + v^2) - 1)  (2v^2 + 1)]

    e = 4//3*ρ*(γ^2 - 1//4) + 2v*γ*Q + v^2*τ
    s = 4//3*ρ*γ^2*v + (v^2+1//1)γ*Q + v*τ;

    Y1 = -3χ₁*γ/T/μ^3 *(2γ^2 - 1//1)
    Y2 = -3χ₁*γ/T/μ^3 *v*(6γ^2 - 1//1)
    Y3 = -3χ₁*γ/T/μ^3 *(6γ^2*v^2 + 1//1)
    
    return  [- con[c[:e]] + 4//3* ρ*(γ^2 - 1//4) + 2//1* Q*γ*v + τ*v^2;
             - con[c[:s]] + 4//3* ρ*γ^2*v + Q*γ*(v^2 + 1//1) + τ*v;
             - con[c[:c1]] + A[1,1]*x1 + A[1,2]*x2 + A[1,3]*x3 + Y1;
             - con[c[:c2]] + A[2,1]*x1 + A[2,2]*x2 + A[2,3]*x3 + Y2;
             - con[c[:c3]] + A[3,1]*x1 + A[3,2]*x2 + A[3,3]*x3 + Y3]
end

F (generic function with 1 method)

Para testear usamos: 
`
flu = [10.0; 0.2; 1.1; 1.5; 1.2]
con = [0.1366314976448222; 0.07009306769467444; 0.06115332989597844; 0.07178418128379448; 0.04927907295689818]
chi = [-1.0; -1.0; -5.0]
`

**Ojo que hay que cambiar el signo a mu!*

In [54]:
flu = [10.0; 0.2; 1.1; 1.5; 1.2]
con = [0.1366314976448222; 0.07009306769467444; 0.06115332989597844; 0.07178418128379448; 0.04927907295689818]
chi = [-1.0; -1.0; -5.0]
@time F(flu,con,chi)

  0.146915 seconds (392.21 k allocations: 20.885 MiB)


5-element Array{Float64,1}:
 -1.6263032587282567e-18
 -1.3877787807814457e-17
 -3.469446951953614e-18
  3.469446951953614e-18
  1.734723475976807e-18

In [4]:
N=5

5

In [5]:
@variables f[1:N], c[1:N], p[1:3]

(Num[f₁, f₂, f₃, f₄, f₅], Num[c₁, c₂, c₃, c₄, c₅], Num[p₁, p₂, p₃])

In [6]:
F(f,c,p);

In [7]:
JS = Symbolics.jacobian(F(f,c,p),f);

In [8]:
J_exp = Symbolics.build_function(JS, f, c, p);

In [9]:
Jac = eval(J_exp[1]);

In [10]:
Jac(rand(5),rand(5),rand(3))

5×5 Array{Float64,2}:
 2.1326e8   -7.36384e6       -2.47609e6  -1.29663e7      -1.62161e6
 1.91586e8  -7.38982e6       -2.37722e6  -1.10918e7      -1.64112e6
 7.87715e6  -8.70425e5  -107976.0        -2.51339e5  -18105.2
 7.43887e6  -8.45607e5   -83779.5        -2.53325e5  -24170.9
 6.54349e6  -8.1213e5    -72674.7        -2.10055e5  -29720.8

In [11]:
function NR_step!(F, Jac, u0, y, p)
    u = u0 - Jac(u0,y, p) \ F(u0,y,p)
end

NR_step! (generic function with 1 method)

In [39]:
p = [-1.0; -1.0; -5.0]
u0 = @SVector[1.0, 0.30, 2.0, 4.0, 5.0]
y = F(u0,[0,0,0,0,0],p)

F(u0.+0.005.*rand(5),y ,p)

#Jac(u0,y .+0.005.*rand(5),p);
F(u0,y .+0.005.*rand(5),p)

5-element Array{Float64,1}:
 -0.00041226333496524603
 -0.0047830657349550165
 -0.004763752786428821
 -0.00011584671176567696
 -3.0158418865333658e-5

In [42]:
ui = @SVector[1.0, 0.30, 3.0, 4.0, 3.0]
NR_step!(F, Jac, u0, y, p)

5-element SArray{Tuple{5},Float64,1,5} with indices SOneTo(5):
 1.0000000000000002
 0.30000000000000004
 2.0000000000000004
 4.000000000000002
 5.000000000000003

In [43]:
u0 = @SVector[1.0, 0.30, 3.0, 4.0, 3.0]
#u0 = [1.0; 0.30; 3.0; 4.0; 3.0]
for i ∈ 1:25
    u0 = NR_step!(F, Jac, u0, y, p)
end
println(u0)
println(F(u0,y,p))

[1.0000000000000002, 0.3000000000000001, 2.0, 4.000000000000002, 5.0]
[-2.000888343900442e-11, -1.8189894035458565e-11, -4.3076653355456074e-14, -2.0161650127192843e-13, -2.007283228522283e-13]


In [78]:
M = 10^3
# it does not work with SMatrix
for j ∈ 1:M
        ui[:,j] = [10.0; 0.2; 1.1; 1.5; 1.2] + 0.1*rand(5)
end
#u0 = SMatrix{N,M}([(i+j)/(N+M) for i ∈ 1:N, j ∈ 1:M])
#u0 = @SMatrix rand(N,M)
#yy = @SMatrix rand(N,M)
#yy = SMatrix{N,M}([0. for i ∈ 1:N, j ∈ 1:M])
yy = zeros(N,M)
for j ∈ 1:M
    yy[:,j] = F(ui[:,j],[0,0,0,0,0],p)
#    yy[:,j] = convert(Array{Float64,1},F(u0[:,j],yy[:,j],p))
end

In [79]:
j = 200
F(ui[:,j],yy[:,j], p)'*F(ui[:,j],yy[:,j], p)

2.8291798035590816e-34

In [80]:
u0 = (ui .- 0.0001 .*rand(N,M))

5×1000 Array{Float64,2}:
 10.048     10.0841    10.0105    …  10.081     10.073     10.0597
  0.205523   0.255179   0.203597      0.209446   0.237672   0.279965
  1.10929    1.15631    1.11421       1.18578    1.18011    1.1229
  1.58835    1.53558    1.5692        1.50319    1.56602    1.58345
  1.25486    1.27512    1.24108       1.28396    1.24105    1.24204

In [82]:
tol = 10^(-8)
iter_max = 5

@time begin
for j ∈ 1:M
    iter = 1
    while F(u0[:,j],yy[:,j], p)'*F(u0[:,j],yy[:,j], p) > tol && iter < iter_max
        u0[:,j] = NR_step!(F, Jac, u0[:,j], yy[:,j], p)
        iter = iter + 1
        if iter == iter_max 
            println("iter_max reached j = $j")
        end
    end
    #println(iter)
end
end

  0.004709 seconds (53.94 k allocations: 4.104 MiB)
